<H1>Clustering the Neighbourhoods of Moscow for identifying potential areas of setting up a Japanese Restaurant</H1>

Importing and downloading the necessary Libraries 

In [0]:

from geopy.geocoders import Nominatim


In [2]:
!pip install geopy


In [3]:
# Install folium
!pip install folium


In [0]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files


import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library


Webscraping the necessary neighbourhood information from Wikipedia

In [0]:
data = requests.get("https://en.wikipedia.org/wiki/Category:Districts_of_Moscow").text

In [0]:

soup = BeautifulSoup(data, 'html.parser')

In [0]:
neighbourhoodList = []

In [0]:
for row in soup.find_all("div", class_="mw-category")[1].findAll("li"):
    neighbourhoodList.append(row.text)

Storing the necessary data into a pandas dataframe

In [49]:
MO_df = pd.DataFrame({"Neighbourhood": neighbourhoodList})

MO_df.head()

,Neighbourhood
0,Aeroport District
1,Akademichesky District
2,"Alexeyevsky District, Moscow"
3,Altufyevsky District
4,"Babushkinsky District, Moscow"


In [0]:
latitudes=[]
longitudes=[]


In [0]:
for i in range(len(MO_df)):
  if (", Moscow" in MO_df.iloc[i,0]):
    MO_df.iloc[i,0]=MO_df.iloc[i,0]+", Russia"
  else:
    MO_df.iloc[i,0]=MO_df.iloc[i,0]+", Moscow, Russia"




In [53]:
MO_df.head()

,Neighbourhood
0,"Aeroport District, Moscow, Russia"
1,"Akademichesky District, Moscow, Russia"
2,"Alexeyevsky District, Moscow, Russia"
3,"Altufyevsky District, Moscow, Russia"
4,"Babushkinsky District, Moscow, Russia"


Extending the dataframe by adding the latitude and longitude of the respective neighbourhoods by using the geopy library

In [0]:
from geopy.extra.rate_limiter import RateLimiter

locator=Nominatim(user_agent='foursquare_agent')
# 1 - conveneint function to delay between geocoding calls
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
# 2- - create location column
MO_df['location'] = MO_df['Neighbourhood'].apply(geocode)
# 3 - create longitude, laatitude and altitude from location column (returns tuple)
MO_df['point'] = MO_df['location'].apply(lambda loc: tuple(loc.point) if loc else None)
# 4 - split point column into latitude, longitude and altitude columns
MO_df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(MO_df['point'].tolist(), index=MO_df.index)


In [60]:
MO_df.head()

,Neighbourhood,location,point,latitude,longitude,altitude
0,"Aeroport District, Moscow, Russia","(район Аэропорт, Северный административный окр...","(55.80050395, 37.5438640931753, 0.0)",55.800504,37.543864,0.0
1,"Akademichesky District, Moscow, Russia","(Академический район, Юго-Западный администрат...","(55.68935945, 37.5779705276754, 0.0)",55.689359,37.577971,0.0
2,"Alexeyevsky District, Moscow, Russia","(Алексеевский район, Северо-Восточный админист...","(55.81104385, 37.6489978088906, 0.0)",55.811044,37.648998,0.0
3,"Altufyevsky District, Moscow, Russia","(Алтуфьевский район, Северо-Восточный админист...","(55.8786951, 37.5867697081122, 0.0)",55.878695,37.586770,0.0
4,"Babushkinsky District, Moscow, Russia","(Бабушкинский район, Северо-Восточный админист...","(55.8671598, 37.6630746992968, 0.0)",55.867160,37.663075,0.0


In [0]:
MO_df.drop(columns=['location','point','altitude'],inplace=True)

In [62]:
MO_df.head()

,Neighbourhood,latitude,longitude
0,"Aeroport District, Moscow, Russia",55.800504,37.543864
1,"Akademichesky District, Moscow, Russia",55.689359,37.577971
2,"Alexeyevsky District, Moscow, Russia",55.811044,37.648998
3,"Altufyevsky District, Moscow, Russia",55.878695,37.586770
4,"Babushkinsky District, Moscow, Russia",55.867160,37.663075


In [0]:
from google.colab import files

MO_df.to_csv('MO_df.csv')


In [83]:
MO_df.shape

(121, 3)

In [0]:
files.download('MO_df.csv')

Making the necessary calls to the FourSquare API to get the venue details pertaining to the neighbourhoods

In [0]:
#four square api credentials 
CLIENT_ID='DFGPDFRW1QYLM2325SQ2JYWUISC4BMIBCOZVNWH0A2JVCMOU'
CLIENT_SECRET='2JDC5HCBRUM3FVDZCHA1GPZEIUIASCZNS0XLF0GCGZBGN3TP'
VERSION='20191224'
LIMIT=100
RADIUS=500


In [0]:
MO_df.dropna(inplace=True)

In [0]:
MO_df.reset_index(inplace=True,drop=True)

In [89]:
MO_df.head()

,Neighbourhood,latitude,longitude
0,"Aeroport District, Moscow, Russia",55.800504,37.543864
1,"Akademichesky District, Moscow, Russia",55.689359,37.577971
2,"Alexeyevsky District, Moscow, Russia",55.811044,37.648998
3,"Altufyevsky District, Moscow, Russia",55.878695,37.586770
4,"Babushkinsky District, Moscow, Russia",55.867160,37.663075


<h1>Folium Map of Moscow Superimposed with neighbourhoods</h1>

In [0]:
MO_lat=55.7558
MO_long=37.6173

In [86]:
MO_map=folium.Map(location=[MO_lat,MO_long],zoom_start=12)
MO_map

In [91]:
MO_map=folium.Map(location=[MO_lat,MO_long],zoom_start=10)
for lat, lng, neighbourhood in zip(MO_df['latitude'],MO_df['longitude'], MO_df['Neighbourhood']):
    label = '{}'.format(neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(MO_map)  
    
MO_map

In [0]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [99]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]



nearby_venues.head()

,name,categories,lat,lng
0,Территория красоты VERY,Salon / Barbershop,55.799240,37.545800
1,Аптека на Старом Зыковском,Pharmacy,55.801950,37.545025
2,Busido,Martial Arts Dojo,55.799815,37.543609
3,HolderHood Coffee,Coffee Shop,55.803040,37.538231
4,Азбука вкуса,Gourmet Shop,55.796894,37.541200


In [100]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

12 venues were returned by Foursquare.


In [0]:
radius = 2000
LIMIT = 100

venues = []

for lat, lng, neighbourhood in zip(MO_df['latitude'], MO_df['longitude'], MO_df['Neighbourhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        lng,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighbourhood,
            lat, 
            lng, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [118]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighbourhood', 'latitude', 'longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(9927, 7)


,Neighbourhood,latitude,longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,"Aeroport District, Moscow, Russia",55.800504,37.543864,Спортзал Mail.Ru,55.797103,37.537994,Gym / Fitness Center
1,"Aeroport District, Moscow, Russia",55.800504,37.543864,Территория красоты VERY,55.799240,37.545800,Salon / Barbershop
2,"Aeroport District, Moscow, Russia",55.800504,37.543864,Cosmic Latte,55.799551,37.534983,Coffee Shop
3,"Aeroport District, Moscow, Russia",55.800504,37.543864,Отдохни,55.805900,37.536928,Liquor Store
4,"Aeroport District, Moscow, Russia",55.800504,37.543864,Аптека на Старом Зыковском,55.801950,37.545025,Pharmacy


In [119]:
venues_df.groupby(["Neighbourhood"]).count()

,latitude,longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighbourhood,,,,,,
"Aeroport District, Moscow, Russia",100,100,100,100,100,100
"Akademichesky District, Moscow, Russia",100,100,100,100,100,100
"Alexeyevsky District, Moscow, Russia",100,100,100,100,100,100
"Altufyevsky District, Moscow, Russia",100,100,100,100,100,100
"Babushkinsky District, Moscow, Russia",100,100,100,100,100,100
"Basmanny District, Moscow, Russia",100,100,100,100,100,100
"Begovoy District, Moscow, Russia",100,100,100,100,100,100
"Beskudnikovsky District, Moscow, Russia",79,79,79,79,79,79
"Bibirevo District, Moscow, Russia",100,100,100,100,100,100


In [120]:

print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 389 uniques categories.


In [121]:
venues_df['VenueCategory'].unique()[:50]

array(['Gym / Fitness Center', 'Salon / Barbershop', 'Coffee Shop',
       'Liquor Store', 'Pharmacy', 'Martial Arts Dojo',
       'Health Food Store', 'Flower Shop', 'Gourmet Shop', 'Juice Bar',
       'Tennis Stadium', 'Deli / Bodega', 'Pet Store', 'Multiplex',
       'Convenience Store', 'Basketball Stadium', 'Massage Studio',
       'Hotel', 'Palace', 'Italian Restaurant', 'Playground',
       'Fast Food Restaurant', 'Shopping Mall', 'Park', 'Cosmetics Shop',
       'Jazz Club', 'Udon Restaurant', 'Skating Rink', 'Gastropub',
       'Hobby Shop', 'Track Stadium', 'Supplement Shop', 'Garden',
       'Hockey Arena', 'Auto Workshop', 'Dessert Shop', 'Arcade',
       'Restaurant', 'Beer Bar', 'Clothing Store', 'Outdoor Gym',
       'Dance Studio', 'Toy / Game Store', 'Gymnastics Gym', 'Aquarium',
       'Ice Cream Shop', 'Board Shop', 'Shoe Store', 'Shop & Service',
       'Kids Store'], dtype=object)

One-Hot encoding the neighbourhood data according to the venue categories and then correspondingly grouping by the neighbourhood

In [122]:

# one hot encoding
MO_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
MO_onehot['Neighbourhoods'] = venues_df['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [MO_onehot.columns[-1]] + list(MO_onehot.columns[:-1])
MO_onehot = MO_onehot[fixed_columns]

print(MO_onehot.shape)
MO_onehot.head()

(9927, 390)


,Neighbourhoods,ATM,Accessories Store,Adult Boutique,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Basketball Stadium,Bath House,Bathing Area,Bay,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Betting Shop,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Blini House,Board Shop,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Camera Store,Canal,Canal Lock,Candy Store,Car Wash,Castle,Caucasian Restaurant,Chinese Restaurant,Church,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Soccer Field,College Track,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Country Dance Club,Coworking Space,Cupcake Shop,Curling Ice,Cycle Studio,Czech Restaurant,Dairy Store,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Dive Bar,Dive Spot,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Film Studio,Fish Market,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Forest,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gun Range,Gun Shop,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,Hill,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hockey Field,Hockey Rink,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hunting Supply,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indoor Play Area,Insurance Office,Irish Pub,Island,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Laser Tag,Leather Goods Store,Lebanese Restaurant,Light Rail Station,Lighthouse,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monastery,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,National Park,Nature Preserve,Neighborhood,Nightclub,Noodle House,Notary,Nudist Beach,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoor Gym,Outdoor Sculpture,Outdoor Supply Store,Outlet Mall,Outlet Store,Paintball Field,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Pelmeni House,Performing Arts Venue,Perfume Shop,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Photography Lab,Photography Studio,Pie Shop,Pier,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Pool Hall,Print Shop,Pub,Public Art,RV Park,Racecourse,Racetrack,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Reservoir,Residential Building (Apartment / Condo),Resort,Rest Area,Restauran

The below MO_grouped dataframe consists of the mean of the onehot encoded data along the neighbourhood as an index 

In [123]:
MO_grouped = MO_onehot.groupby(["Neighbourhoods"]).mean().reset_index()

print(MO_grouped.shape)
MO_grouped

(120, 390)


,Neighbourhoods,ATM,Accessories Store,Adult Boutique,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Basketball Stadium,Bath House,Bathing Area,Bay,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Betting Shop,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Blini House,Board Shop,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Camera Store,Canal,Canal Lock,Candy Store,Car Wash,Castle,Caucasian Restaurant,Chinese Restaurant,Church,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Soccer Field,College Track,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Country Dance Club,Coworking Space,Cupcake Shop,Curling Ice,Cycle Studio,Czech Restaurant,Dairy Store,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Dive Bar,Dive Spot,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Film Studio,Fish Market,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Forest,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gun Range,Gun Shop,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,Hill,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hockey Field,Hockey Rink,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hunting Supply,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indoor Play Area,Insurance Office,Irish Pub,Island,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Laser Tag,Leather Goods Store,Lebanese Restaurant,Light Rail Station,Lighthouse,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monastery,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,National Park,Nature Preserve,Neighborhood,Nightclub,Noodle House,Notary,Nudist Beach,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoor Gym,Outdoor Sculpture,Outdoor Supply Store,Outlet Mall,Outlet Store,Paintball Field,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Pelmeni House,Performing Arts Venue,Perfume Shop,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Photography Lab,Photography Studio,Pie Shop,Pier,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Pool Hall,Print Shop,Pub,Public Art,RV Park,Racecourse,Racetrack,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Reservoir,Residential Building (Apartment / Condo),Resort,Rest Area,Restauran

Looking at only the "JAPANESE RESTAURANT" in accordance with the requirement

In [0]:

MO_HOTEL = MO_grouped[["Neighbourhoods","Japanese Restaurant"]]

In [180]:
MO_HOTEL.shape


(120, 2)

defining the number of clusters and fitting the model to the data

In [181]:
# set number of clusters
kclusters = 4

MO_clustering = MO_HOTEL.drop(["Neighbourhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(MO_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 2, 2, 0, 1, 1, 3, 2, 0], dtype=int32)

Merging the label values with the above dataframe

In [0]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
MO_merged = MO_HOTEL.copy()

# add clustering labels
MO_merged["Cluster Labels"] = kmeans.labels_


In [183]:
MO_merged.rename(columns={"Neighbourhoods": "Neighbourhood"}, inplace=True)
MO_merged.head()

,Neighbourhood,Japanese Restaurant,Cluster Labels
0,"Aeroport District, Moscow, Russia",0.00,1
1,"Akademichesky District, Moscow, Russia",0.00,1
2,"Alexeyevsky District, Moscow, Russia",0.01,2
3,"Altufyevsky District, Moscow, Russia",0.01,2
4,"Babushkinsky District, Moscow, Russia",0.02,0


In [184]:
# merge MO_grouped with MO_df to add latitude/longitude for each neighborhood
MO_merged = MO_merged.join(MO_df.set_index("Neighbourhood"), on="Neighbourhood")

print(MO_merged.shape)
MO_merged.head() # check the last columns!

(120, 5)


,Neighbourhood,Japanese Restaurant,Cluster Labels,latitude,longitude
0,"Aeroport District, Moscow, Russia",0.00,1,55.800504,37.543864
1,"Akademichesky District, Moscow, Russia",0.00,1,55.689359,37.577971
2,"Alexeyevsky District, Moscow, Russia",0.01,2,55.811044,37.648998
3,"Altufyevsky District, Moscow, Russia",0.01,2,55.878695,37.586770
4,"Babushkinsky District, Moscow, Russia",0.02,0,55.867160,37.663075


In [185]:
# sort the results by Cluster Labels
print(MO_merged.shape)
MO_merged.sort_values(["Cluster Labels"], inplace=True)
MO_merged

(120, 5)


,Neighbourhood,Japanese Restaurant,Cluster Labels,latitude,longitude
65,"Novokosino District, Moscow, Russia",0.017241,0,55.739857,37.860310
17,"Cheryomushki District, Moscow, Russia",0.020000,0,55.665151,37.564752
16,"Chertanovo Yuzhnoye District, Moscow, Russia",0.020000,0,55.589835,37.595748
68,"Orekhovo-Borisovo Severnoye District, Moscow, ...",0.020000,0,55.620866,37.703759
35,"Kosino-Ukhtomsky District, Moscow, Russia",0.020833,0,55.713562,37.882080
102,"Tyoply Stan District, Moscow, Russia",0.020000,0,55.625932,37.493834
34,"Koptevo District, Moscow, Russia",0.020000,0,55.830065,37.521738
44,"Levoberezhny District, Moscow, Russia",0.020000,0,55.865663,37.465859
71,"Otradnoye District, Moscow, Russia",0.020833,0,55.863422,37.603802
38,"Krylatskoye District, Moscow, Russia",0.020000,0,55.761287,37.425934


Visualing the clusters

In [186]:
# create map
MO_lat=55.7558
MO_long=37.6173
map_clusters = folium.Map(location=[MO_lat,MO_long], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(MO_merged['latitude'], MO_merged['longitude'], MO_merged['Neighbourhood'], MO_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [0]:
map_clusters.save('map_clusters.html')

In [187]:
MO_merged.loc[MO_merged['Cluster Labels'] == 0]


,Neighbourhood,Japanese Restaurant,Cluster Labels,latitude,longitude
65,"Novokosino District, Moscow, Russia",0.017241,0,55.739857,37.860310
17,"Cheryomushki District, Moscow, Russia",0.020000,0,55.665151,37.564752
16,"Chertanovo Yuzhnoye District, Moscow, Russia",0.020000,0,55.589835,37.595748
68,"Orekhovo-Borisovo Severnoye District, Moscow, ...",0.020000,0,55.620866,37.703759
35,"Kosino-Ukhtomsky District, Moscow, Russia",0.020833,0,55.713562,37.882080
102,"Tyoply Stan District, Moscow, Russia",0.020000,0,55.625932,37.493834
34,"Koptevo District, Moscow, Russia",0.020000,0,55.830065,37.521738
44,"Levoberezhny District, Moscow, Russia",0.020000,0,55.865663,37.465859
71,"Otradnoye District, Moscow, Russia",0.020833,0,55.863422,37.603802
38,"Krylatskoye District, Moscow, Russia",0.020000,0,55.761287,37.425934


In [188]:
MO_merged.loc[MO_merged['Cluster Labels'] == 1]


,Neighbourhood,Japanese Restaurant,Cluster Labels,latitude,longitude
74,"Pokrovskoye-Streshnevo District, Moscow, Russia",0.0,1,55.825025,37.450401
76,"Presnensky District, Moscow, Russia",0.0,1,55.762215,37.560658
73,"Perovo District, Moscow, Russia",0.0,1,55.751069,37.753163
0,"Aeroport District, Moscow, Russia",0.0,1,55.800504,37.543864
67,"Ochakovo-Matveyevskoye District, Moscow, Russia",0.0,1,55.692105,37.440192
70,"Ostankinsky District, Moscow, Russia",0.0,1,55.821077,37.619119
78,"Ramenki District, Moscow, Russia",0.0,1,55.701835,37.504107
66,"Obruchevsky District, Moscow, Russia",0.0,1,55.657643,37.520459
61,"Nekrasovka District, Moscow, Russia",0.0,1,55.697445,37.942282
58,"Nagatino-Sadovniki District, Moscow, Russia",0.0,1,55.671858,37.644623


In [189]:
MO_merged.loc[MO_merged['Cluster Labels'] == 2]


,Neighbourhood,Japanese Restaurant,Cluster Labels,latitude,longitude
8,"Bibirevo District, Moscow, Russia",0.010000,2,55.895319,37.612989
109,"Vykhino-Zhulebino District, Moscow, Russia",0.014085,2,55.698397,37.824332
98,"Timiryazevsky District, Moscow, Russia",0.012500,2,55.825817,37.557744
97,"Tekstilshchiki District, Moscow, Russia",0.010000,2,55.703181,37.740592
3,"Altufyevsky District, Moscow, Russia",0.010000,2,55.878695,37.586770
2,"Alexeyevsky District, Moscow, Russia",0.010000,2,55.811044,37.648998
95,"Sviblovo District, Moscow, Russia",0.010870,2,55.852684,37.647132
105,"Vostochnoye Degunino District, Moscow, Russia",0.011494,2,55.875872,37.563295
115,"Yuzhnoye Medvedkovo District, Moscow, Russia",0.011765,2,55.874826,37.633894
117,"Zapadnoye Degunino District, Moscow, Russia",0.011628,2,55.870548,37.520804


In [190]:
MO_merged.loc[MO_merged['Cluster Labels'] == 3]


,Neighbourhood,Japanese Restaurant,Cluster Labels,latitude,longitude
7,"Beskudnikovsky District, Moscow, Russia",0.025316,3,55.863739,37.557322
118,"Zyablikovo District, Moscow, Russia",0.027397,3,55.621107,37.746476
48,"Lyublino District, Moscow, Russia",0.027027,3,55.674310,37.782028
114,"Yuzhnoye Butovo District, Moscow, Russia",0.035714,3,55.541272,37.532539
56,"Moskvorechye-Saburovo District, Moscow, Russia",0.024691,3,55.643215,37.659510
69,"Orekhovo-Borisovo Yuzhnoye District, Moscow, R...",0.030000,3,55.603957,37.726930


It can be seen from the above results and visualisations that cluster 0 and 3 cover least number of neighbourhoods suggesting that they offer little to no competetion in terms of setting up a japanese restaurant , also it can be logically understood that the other clusters surrounds mostly the central and near-central moscow , suggesting that there are many established restaurants at the centre of the city , maybe running from years altogether and hence leaving out little space and/or vis-a-vis high property rates that have pushed the other resturants that can be setup at the other ends of the city , which cluster 3 suggests clearly.Hence it is advisable to setup a restaurant in above mentioned clusters in order to avoid cut-throat competition and also helping serve the purpose of lack of japanese restaurants in the above cluster, which can henceforth become the centre of attraction eventually.